In [15]:
import pandas as pd
import numpy as np
import datetime as dt
import os

DATA_DIR = 'data'
CSV_DIR = 'csv'

In [16]:
# Reading in data
read_path = os.path.join(DATA_DIR, CSV_DIR, 'technicals.csv')
data = pd.read_csv(read_path)

In [17]:
# Creating buy and sell signals
data['call_buy'] = np.where((data['low'] <= data['lower_band']) & (data['time'] > '10:30:00') & (data['call_setup'] == True), True, False)
data['call_sell'] = np.where((data['high'] >= data['ema_20']) & (data['time'] > "10:30:00"), True, False)

data['put_buy'] = np.where((data['high'] >= data['upper_band']) & (data['time'] > '10:30:00') & (data['put_setup'] == True), True, False)
data['put_sell'] = np.where((data['low'] <= data['ema_20']) & (data['time'] > "10:30:00"), True, False)

In [18]:
# Removing times when buy and sell are at the same time
data = data[(data['call_buy'] & data['call_sell']) != True].copy()
data = data[(data['put_buy'] & data['put_sell']) != True].copy()

data = data[(data['call_buy'] | data['call_sell'] | data['put_buy'] | data['put_sell']) == True].copy()

In [19]:
# Initializing variabel for last eaction
last = None

for index, row in data.iterrows():
    """
    Looping through all rows using logic to determine what vlaues each row should have based on the vlaue of the last action
    Results in data frame such that if call_buy = True than the next True value will be in the call_sell column
    and if put_buy = True than the next True value will be in the put_sell column
    """
    if row['call_buy']:
        if last == 'put_buy':
            data.at[index, 'call_buy'] = False
            data.at[index, 'call_sell'] = False
        
        elif last == 'call_buy':
            data.at[index, 'call_buy'] = False
            data.at[index, 'put_buy'] = False
            data.at[index, 'put_sell'] = False

        else:
            last = 'call_buy'
            data.at[index, 'put_sell'] = False

    if row['put_buy']:
        if last == 'put_buy':
            data.at[index, 'call_buy'] = False
            data.at[index, 'call_sell'] = False
            data.at[index, 'put_buy'] = False
        
        elif last == 'call_buy':
            data.at[index, 'put_sell'] = False
            data.at[index, 'put_buy'] = False

        else:
            last = 'put_buy'
            data.at[index, 'call_sell'] = False

    if row['call_sell']:
        if last == 'call_buy':
            last = 'call_sell'
            data.at[index, 'put_buy'] = False

        else:
            data.at[index, 'call_sell'] = False

    if row['put_sell']:
        if last == 'put_buy':
            last = 'put_sell'
            data.at[index, 'call_buy'] = False

        else:
            data.at[index, 'put_sell'] = False


In [20]:
# Clearing dataframe of uneeded data
data = data[(data['call_buy'] | data['call_sell'] | data['put_buy'] | data['put_sell']) == True].copy()

In [21]:
# Saving dataframe to csv
save_path = os.path.join(DATA_DIR, CSV_DIR, 'signals.csv')
data.to_csv(save_path, index=False)
data = None